In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def fix_zipcodes(zipcode):
    if len(zipcode) < 5:
        to_fill = 5 - len(zipcode)
        return (to_fill * '0') + zipcode

    else:
        return zipcode

def fix_county_name(county_name):
    if 'county' in county_name:
        return county_name[:county_name.find('county')].rstrip()
    else:
        return county_name

def match_loc_to_ideology(zipcode):
    state = ZIPS_DF[ZIPS_DF['zip'] == zipcode]['state'].iloc[0]
    county_name = fix_county_name(ZIPS_DF[ZIPS_DF['zip'] == zipcode]['county'].iloc[0])

    state_slice = COUNTY_DF[COUNTY_DF['state_po'] == state]
    ideology_metric = state_slice.loc[state_slice['county_name']==county_name]['perc_diffs'].iloc[0]

    return ideology_metric

def process_cpvi(cpvi):
    if cpvi.startswith('R'):
        return int(cpvi.split('+')[1])
    elif cpvi.startswith('D'):
        cpvi_metric = int(cpvi.split('+')[1])
        return np.negative(cpvi_metric)
    else:
        return 0

In [3]:
BUSINESS_DATA_FILEPATH = 'data/yelp_academic_dataset_business.json'
REVIEWS_DATA_FILEPATH = 'data/yelp_academic_dataset_review.json'
USERS_DATA_FILEPATH = 'data/yelp_academic_dataset_user.json'
COUNTY_DATA_FILEPATH = 'data/county_data.csv'
STATES_DATA_FILEPATH = 'data/states_data.csv'
ZIPCODES_DATA_FILEPATH = 'data/zip_code_database.csv'
CPI_URL = 'https://en.wikipedia.org/wiki/Cook_Partisan_Voting_Index'

# Guide

The goal is to put everything into one table in tabular format.

# Plan

## Location tracking

1. Query each review for the business ID to get long and lat values.
2. Take average of long and lat values for a centered location.
3. Take values by users.
4. Assign political ideologies by county and state.
5. Count helpfulness by user.

## Scrap the location tracking old plan

1. For each user, count the zip code of the business reviewed
2. Assign it to a county
3. The most common county is the user's base location

## Political ideology

1. Match with county & state levels
2. Cook Partisan Voting Index - review data and revisit

## Business & Reviews

1. Keep zip code
2. Assign categorization: restaurants, home services, auto services, other
3. Assign a size
4. Assign the state
5. Business mean rating
6. Helpfulness count
7. Review rating

In [4]:
COUNTY_DF = pd.read_csv(COUNTY_DATA_FILEPATH, index_col=0)
STATES_DF = pd.read_csv(STATES_DATA_FILEPATH, index_col=0)
ZIPS_DF = pd.read_csv(ZIPCODES_DATA_FILEPATH)

In [5]:
ZIPS_DF = ZIPS_DF[['zip', 'state', 'county']]
ZIPS_DF['zip'] = ZIPS_DF['zip'].astype(str).apply(fix_zipcodes)
ZIPS_DF['county'] = ZIPS_DF['county'].str.lower()
ZIPS_DF.loc[len(ZIPS_DF)] = ['02101', 'MA', 'suffolk county']
ZIPS_DF.head()

,zip,state,county
0,00501,NY,suffolk county
1,00544,NY,suffolk county
2,00601,PR,adjuntas municipio
3,00602,PR,aguada municipio
4,00603,PR,aguadilla municipio


In [6]:
COUNTY_DF['county_name'] = COUNTY_DF['county_name'].str.lower()
COUNTY_DF['county_name'] = COUNTY_DF['county_name'].apply(fix_county_name)
COUNTY_DF.head()

,state,state_po,county_name,perc_diffs
50526,ALABAMA,AL,autauga,44.42
50529,ALABAMA,AL,baldwin,53.76
50532,ALABAMA,AL,barbour,7.66
50535,ALABAMA,AL,bibb,57.73
50538,ALABAMA,AL,blount,80.00


In [7]:
COUNTY_DF[COUNTY_DF['state_po'] == 'CA'].sample(10)

,state,state_po,county_name,perc_diffs
52754,CALIFORNIA,CA,madera,11.56
52904,CALIFORNIA,CA,stanislaus,-0.79
52899,CALIFORNIA,CA,sonoma,-51.48
52704,CALIFORNIA,CA,fresno,-7.83
52804,CALIFORNIA,CA,orange,-9.04
52774,CALIFORNIA,CA,merced,-10.59
52944,CALIFORNIA,CA,yuba,21.63
52919,CALIFORNIA,CA,trinity,5.38
52819,CALIFORNIA,CA,riverside,-7.94
52844,CALIFORNIA,CA,san francisco,-72.54


In [8]:
STATES_DF.head()

,state,perc_diffs
1,ALABAMA,25.46
3,ALASKA,10.11
5,ARIZONA,-0.10
7,ARKANSAS,6.91
9,CALIFORNIA,-29.16


In [9]:
STATES_LIST = list(COUNTY_DF['state_po'].unique())
print(f"Number of states: {len(STATES_LIST)}")

Number of states: 51


In [10]:
CPVI = pd.read_html(CPI_URL)[1]
CPVI.drop(CPVI.tail(1).index, inplace=True)
CPVI['pvi'] = CPVI['PVI'].apply(process_cpvi)
CPVI['state'] = CPVI['State'].str.upper()
CPVI = CPVI[['state', 'pvi']]

In [11]:
USER_COLS = ['user_id', 'review_count']
BUSINESS_COLS = ['business_id', 'state', 'city', 'postal_code', 'categories', 'stars', 'review_count']
REVIEW_COLS = ['review_id', 'user_id', 'business_id', 'date', 'stars', 'useful']
ZIPCODES_COLS = ['zip', 'county']

USER_DTYPES = {
    'user_id': np.str,
    'review_count': np.int
}

BUSINESS_DTYPES = {
    'business_id': np.str,
    'state': np.str,
    'city': np.str,
    'postal_code': np.str,
    'categories': np.str,
    'review_count': np.int,
    'stars': np.float
}

REVIEW_DTYPES = {
    'review_id': np.str,
    'user_id': np.str,
    'business_id': np.str,
    'stars': np.int,
    'useful': np.int
}

ZIPS_DTYPES = {
    'zip': np.str,
    'county': np.str
}

In [12]:
# Load 20,000 reviews from reviews
REVIEWS = list()
with open(REVIEWS_DATA_FILEPATH, 'r') as f:
    reader = pd.read_json(f, orient='records', lines=True, nrows=300000, chunksize=1000, dtype=REVIEW_DTYPES)

    for chunk in tqdm(reader):
        reduced_chunk = chunk[REVIEW_COLS]
        # Only keep US
        REVIEWS.append(reduced_chunk)
    REVIEWS = pd.concat(REVIEWS, ignore_index=True)

300it [00:04, 61.07it/s]


In [13]:
REVIEWS.sample(15)

,review_id,user_id,business_id,date,stars,useful
68708,6bN3Jkx5iuP72ViLf64amQ,GSm7mzKgvwReL4xVUi_c8w,6GczIaxfVePcP1ztujRisw,2010-07-08 22:05:23,2,1
116277,zM5q0FPwonDznY-FcAmrwg,0vXoxwVuieH0f0h-FeU_Gw,5I0yJ_X7uyPmoJWzKRYDuA,2018-07-11 14:53:11,3,0
147515,FvSMfDnTmlJJj1rDnpF-rw,1jLThvvGOqxZGGqmadXAPg,Wm3pOIWzjzttU7TgGGyM7Q,2018-03-06 20:13:13,4,0
18546,qFL2DaY5R8krLLqUoUjXeA,IPK5MAhmgdjIm5jB-j5ebQ,Yw1vQd5U5GB6ZPqn2buhBg,2018-04-28 02:35:06,1,0
285185,Ng9KZtE9ZuID41DWOo2qeQ,LRKDNfB3Wfp0HEYmbsyw5A,880YPPlGyi_49AXLH8xIeA,2011-08-11 18:29:43,5,1
173506,rDYvwcpFvkLM1dqE3YqnpQ,KxT2YZMCg9KmwC6mJKjgSw,d5os_dzoyEloK_zsVF0pYw,2014-09-15 14:31:04,2,5
108430,xupIiHcCa3TyGDk3Rxpk6g,Botcqvdafz1975sIhM2Uyg,_cxOC-jzriK2zcQj2QxBcw,2017-03-06 18:44:18,5,0
81541,T6C46zwL5bjwM4ge4xQxAQ,fmQmCBGw0OkekcUf9wlq1w,0iqIdvlZpXwnyRVNQ1dgkA,2014-11-15 18:05:45,4,0
179978,uql1xurYj81sIbP7ud2rFA,_O1uA-NXH4myeF6pWFcuBA,8IGgRKegd0H287W66X0BOw,2017-08-11 19:32:26,1,0
161878,Lbvs7qcYQFt2Y974-qD7hA,KJmXsA1zFPH-n6r14JcYLw,YCuGxnZvDl1JFROkMS3kwg,2015-09-27 14:07:48,3,2


In [14]:
USERS_LIST = list(REVIEWS['user_id'].unique())
BUSINESS_LIST = list(REVIEWS['business_id'].unique())


In [15]:
BUSINESSES = list()

with open(BUSINESS_DATA_FILEPATH, 'r') as f:
    reader = pd.read_json(f, orient='records', lines=True, chunksize=1000, dtype=BUSINESS_DTYPES)

    for chunk in tqdm(reader):
        reduced_chunk = chunk[BUSINESS_COLS]
        reduced_chunk = reduced_chunk[reduced_chunk['business_id'].isin(BUSINESS_LIST)]
        BUSINESSES.append(reduced_chunk)

    BUSINESSES = pd.concat(BUSINESSES, ignore_index=True)

print("Businesses loaded.")

161it [00:04, 33.85it/s]

Businesses loaded.


In [16]:
BUSINESSES.sample(10)


,business_id,state,city,postal_code,categories,stars,review_count
8726,U_aL7LMoxaa8wbD8eaEeIg,MA,Wakefield,01880,"Bartenders, Caterers, Event Planning & Services",4.5,7
5048,OIVKobtZLAmVB1dw9vMXpg,GA,Atlanta,30363,"Shopping, Flowers & Gifts, Florists",4.5,52
13662,PmS4Fki9AATqX7SA0CrdUw,MA,Boston,02114,"Education, Professional Services, Test Prepara...",3.5,10
9163,KK7wFlO7Nn6H3Iz1-bEB3g,TX,Austin,78745,"Security Systems, Local Services, Electricians...",4.0,9
8485,lkSsBChJCFkLrvlDGTv0pw,MA,Cambridge,02238,"Physical Therapy, Health & Medical",3.0,9
11364,LuAjpZ4p2HxyczLNSpoufQ,TX,Austin,78749,"Food, Beer, Wine & Spirits, Shopping, Wholesal...",4.0,5
5688,IrdCTkknOaNg0AXjA-KQJQ,TX,Austin,78744,"Home Services, Painters, Drywall Installation ...",3.5,8
8378,ex7nYhK8oB2owQ2jPNjM2w,GA,Atlanta,30309,"American (New), Restaurants, Sandwiches",3.5,101
6990,zGUxnIJM8v1QhJ7aQXjHvw,TX,Austin,78745,"Vietnamese, Restaurants, Taiwanese, Chinese",2.0,55
11903,zfUWCi69OaX2g5P8ZbfqNQ,GA,Sandy Springs,30328,"Chiropractors, Naturopathic/Holistic, Physical...",4.5,16


In [17]:
# add business zip code in first
# add business state
print('Adding business state')
for business_id in tqdm(BUSINESS_LIST):
    business_zipcode = BUSINESSES[BUSINESSES['business_id'] == business_id]['postal_code'].iloc[0]
    business_state = BUSINESSES[BUSINESSES['business_id'] == business_id]['state'].iloc[0]

    REVIEWS.loc[REVIEWS['business_id'] == business_id, ['business_state', 'business_zipcode']] = business_state, business_zipcode

REVIEWS = REVIEWS[REVIEWS['business_state'].isin(STATES_LIST)]

100%|██████████| 13706/13706 [09:33<00:00, 23.91it/s]


In [18]:
# assign business political ideologies by zip code
BUSINESS_ZIPS = list(REVIEWS['business_zipcode'].unique())
print('Adding business zipcode ideology')
for business_zip in tqdm(BUSINESS_ZIPS):
    try:
        business_ideology = match_loc_to_ideology(business_zip)
        REVIEWS.loc[REVIEWS['business_zipcode'] == business_zip, 'business_zipcode_ideology'] = business_ideology
    except:
        print(business_zip)

  1%|          | 4/533 [00:00<00:13, 39.66it/s]

Adding business zipcode ideology


  3%|▎         | 17/533 [00:00<00:12, 40.36it/s]

100%|██████████| 533/533 [00:13<00:00, 40.30it/s]

97226


In [19]:
len(USERS_LIST)

204997

In [20]:
STATES_ABBR = dict()

for state in list(STATES_DF['state']):
    STATES_ABBR[state] = COUNTY_DF[COUNTY_DF['state'] == state]['state_po'].unique()[0]

print(STATES_ABBR)

{'ALABAMA': 'AL', 'ALASKA': 'AK', 'ARIZONA': 'AZ', 'ARKANSAS': 'AR', 'CALIFORNIA': 'CA', 'COLORADO': 'CO', 'CONNECTICUT': 'CT', 'DELAWARE': 'DE', 'DISTRICT OF COLUMBIA': 'DC', 'FLORIDA': 'FL', 'GEORGIA': 'GA', 'HAWAII': 'HI', 'IDAHO': 'ID', 'ILLINOIS': 'IL', 'INDIANA': 'IN', 'IOWA': 'IA', 'KANSAS': 'KS', 'KENTUCKY': 'KY', 'LOUISIANA': 'LA', 'MAINE': 'ME', 'MARYLAND': 'MD', 'MASSACHUSETTS': 'MA', 'MICHIGAN': 'MI', 'MINNESOTA': 'MN', 'MISSISSIPPI': 'MS', 'MISSOURI': 'MO', 'MONTANA': 'MT', 'NEBRASKA': 'NE', 'NEVADA': 'NV', 'NEW HAMPSHIRE': 'NH', 'NEW JERSEY': 'NJ', 'NEW MEXICO': 'NM', 'NEW YORK': 'NY', 'NORTH CAROLINA': 'NC', 'NORTH DAKOTA': 'ND', 'OHIO': 'OH', 'OKLAHOMA': 'OK', 'OREGON': 'OR', 'PENNSYLVANIA': 'PA', 'RHODE ISLAND': 'RI', 'SOUTH CAROLINA': 'SC', 'SOUTH DAKOTA': 'SD', 'TENNESSEE': 'TN', 'TEXAS': 'TX', 'UTAH': 'UT', 'VERMONT': 'VT', 'VIRGINIA': 'VA', 'WASHINGTON': 'WA', 'WEST VIRGINIA': 'WV', 'WISCONSIN': 'WI', 'WYOMING': 'WY'}


In [21]:
print('Processing business state ideology and state pvi')
for state in tqdm(list(STATES_ABBR.keys())):
    try:
        state_ideology = STATES_DF[STATES_DF['state'] == state]['perc_diffs'].iloc[0]

        if state == 'DISTRICT OF COLUMBIA':
            state_pvi = 0
        else:
            state_pvi = CPVI[CPVI['state'] == state]['pvi'].iloc[0]

        REVIEWS.loc[REVIEWS['business_state'] == STATES_ABBR[state],
                    ['bussiness_state_ideology', 'business_state_pvi']] = state_ideology, state_pvi
    except:
        print(state)
        print('bad output')

  0%|          | 0/51 [00:00<?, ?it/s]

Processing business state ideology and state pvi


100%|██████████| 51/51 [00:00<00:00, 51.64it/s]


In [25]:
print('Adding total reviews and average stars per business')
REVIEWS['count'] = 1
BUSINESS_COUNTS = REVIEWS.groupby('business_id').sum().reset_index()[['business_id', 'count']]
BUSINESS_STARS = REVIEWS[['business_id', 'stars']].groupby('business_id').sum().reset_index()
BUSINESSES_LIST = list(BUSINESS_COUNTS['business_id'].unique())

for business in tqdm(BUSINESSES_LIST):
    tot_reviews = BUSINESS_COUNTS[BUSINESS_COUNTS['business_id'] == business]['count'].iloc[0]
    avg_stars = round((BUSINESS_STARS[BUSINESS_STARS['business_id'] == business]['stars'].iloc[0] / tot_reviews), 3)
    REVIEWS.loc[REVIEWS['business_id'] == business, ['business_review_total', 'avg_star_rating']] = tot_reviews, avg_stars


Adding total reviews and average stars per business


100%|██████████| 12212/12212 [05:36<00:00, 36.28it/s]


In [26]:
REVIEWS.drop(columns=['count'], inplace=True)
REVIEWS

,review_id,user_id,business_id,date,stars,useful,business_state,business_zipcode,business_zipcode_ideology,bussiness_state_ideology,business_state_pvi,business_review_total,avg_star_rating
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,2014-10-11 03:34:02,4,3,MA,01915,-28.82,-33.21,-14.0,44.0,3.614
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,2015-07-03 20:38:25,4,1,MA,01701,-44.89,-33.21,-14.0,23.0,3.696
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,2013-05-28 20:38:06,5,0,OR,97210,-61.31,-16.08,-6.0,96.0,4.760
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2010-01-08 02:29:15,2,1,FL,32821,-23.12,3.36,3.0,121.0,3.446
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,2011-07-28 18:05:01,4,0,CO,80302,-56.57,-13.50,-3.0,472.0,4.282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,I0mRlZcCGAyntyGclLHSIw,vso8sQvCRVUnegGY9fx5rQ,Q4DRuxHhKuxP-01rYiRJnQ,2017-02-03 00:45:08,5,0,OR,97086,-11.07,-16.08,-6.0,47.0,2.766
299996,f_dYCOA9_zadlLtzWA2DWw,suv1g1uAHm0DQu_BMmPznA,sLkK9Ofmk9UFDrjXxh3iLA,2014-12-07 17:26:02,5,0,OR,97227,-61.31,-16.08,-6.0,16.0,4.562
299997,O0SG6BTN0FHSZ7PJ1wgmzg,mVxg_vBXcUAAQJ03uHxiIQ,La_xmePMtWwjxKUqbGvqGQ,2010-05-30 11:06:52,2,0,TX,78703,-45.11,5.58,5.0,108.0,3.944
299998,vTovPnBesDco238QpKtAvA,uVpNSrc3IEpR42VGdZ709w,4jOreXu9ctLBWOJWMtNv1w,2017-01-07 05:55:39,4,0,WA,98665,-5.09,-19.20,-8.0,18.0,3.111


In [27]:
REVIEWS.to_csv('data/yelp_dataset.csv')